In [30]:
import sys
sys.path.insert(0,"..") # salgebra is here

from dalgebra import *
%display latex

In [71]:
n = 3; m = 3
R = DifferentialPolynomialRing(QQ, [f"u{i}" for i in range(n-1)] + [f"Y{j}" for j in range(m-1)] + ["z"])
Y, u, z = R.gens()[:m-1], R.gens()[m-1:m+n-2], R.gens()[-1] ;
Y,u,z

((Y0_*, Y1_*), (u0_*, u1_*), z_*)

In [72]:
L = z[n] + sum(z[i]*u[i][0] for i in range(n-1))
P = z[m] + sum(z[i]*Y[i][0] for i in range(m-1))

In [73]:
L

u0_0*z_0 + u1_0*z_1 + z_3

In [74]:
P

Y0_0*z_0 + Y1_0*z_1 + z_3

In [75]:
commutator = L(z=P) - P(z=L); commutator

Y0_1*u1_0*z_0 + Y1_1*u1_0*z_1 - Y1_0*u0_1*z_0 - Y1_0*u1_1*z_1 + Y0_3*z_0 + 3*Y0_2*z_1 + 3*Y0_1*z_2 + Y1_3*z_1 + 3*Y1_2*z_2 + 3*Y1_1*z_3 - u0_3*z_0 + (-3)*u0_2*z_1 + (-3)*u0_1*z_2 - u1_3*z_1 + (-3)*u1_2*z_2 + (-3)*u1_1*z_3

In [76]:
for i in range(commutator.order(z)+1):
    print(f"{i} ---> {commutator.coefficient(z[i])}")

0 ---> Y0_1*u1_0 - Y1_0*u0_1 + Y0_3 - u0_3
1 ---> Y1_1*u1_0 - Y1_0*u1_1 + 3*Y0_2 + Y1_3 + (-3)*u0_2 - u1_3
2 ---> 3*Y0_1 + 3*Y1_2 + (-3)*u0_1 + (-3)*u1_2
3 ---> 3*Y1_1 + (-3)*u1_1


In [77]:
DifferentialSystem([commutator.coefficient(z[i]) for i in range(n-1, commutator.order(z)+1)], variables=Y)

System over [Ring of operator polynomials in (Y0, Y1, u0, u1, z) over Differential Ring [[Rational Field], (0,)]] with variables [(Y0_*, Y1_*)]:
{
	3*Y0_1 + 3*Y1_2 + (-3)*u0_1 + (-3)*u1_2 == 0
	3*Y1_1 + (-3)*u1_1 == 0
}

In [ ]:
y_0 = (2/3)*u_1 + c

$P_2 = c_2 + (2/3)u_1 + \partial^2$

$Y_0 + (u_1 + c)\partial + \partial^3 = (Y_0 + u_1\partial + \partial^3) + cP_1$